# SCRIPT 01 - Creating Training Samples to Separate Between Annual and Perennial Agriculture

This script aims at helping the used create appropriate training samples, in order to make a separation between annual and perennial agriculture, considering a previously created agriculture mask. More about the algorithms employed can be read before each cell.

In this cell needed libraries are imported, namely:

+ `rasterio`
    + Read raster files (.tif) like the references, as well as used to extract the time series data.
+ `numpy`
    + Library used mainly to define matrix structures used to process and store the reference data.
+ `matplotlib.pyplot`
    + Used mainly to plot data and help better visualize the dimensionality of data created. Important to help notice problems during processing.
+ `geopandas`
    + Library used to load vector files, like the tiles definition file.
+ `shapely.geometry`
    + Library from which `Point` is imported, a construct used to save points into the shapefile format. Used to store the location of each training sample.
+ `os`
    + Utilitarian library used mostly to check file paths.

In [ ]:
import rasterio as r
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import geopandas as gpd
from shapely.geometry import Point
import os

Here are defined the parameters for training samples creation. Each one must be taken into consideration carefully according to its requirements.

+ `num_samples` (`int`)
    + The number of samples for each class. If 50,000 is chosen, for example, in the end a total of 100,000 samples will be created because of the two classes used.
+ `samples_id` (`string`)
    + An arbitrary id chosen by the user to help separate groups of samples from others. This in embedded into the file names, so it should not include inappropriate characters to file paths, like /, *, among others.
+ `path_reference` (`string`)
    + A string path to the reference data to be used. This should be in raster format (.tif) and have the exact same crs, extent, and amount of pixels as the agriculture mask. No data value should be 0, annual agriculture must be stored as 1, and perennial agriculture must be 2.
+ `path_agriculture_mask` (`string`)
    + A string path to the agriculture mask created in previous steps. This should be in raster format (.tif) and have the exact same crs, extent, and amount of pixels as the reference for annual and perennial agriculture. No data value should be 0, not agriculture should be 1, and agriculture must be 2.
+ `path_to_tiles_grid` (`string`)
    + A string path to the tiles definition file, in vector format (.shp). This file was used in the previous method to define tiles used to create the time series in Google Earth Engine. It should have a field `id` correspondent to each tile. It is used to decide from which tile a time series value should be extracted.
+ `path_to_monthly_time_series_folder` (`string`)
    + A string path to the folder where the monthly time series are stored. Each tile of time series must be named according to the following format: `Reduction_Optical_Months_id{tile_id:03}_{band_name_}.tif`. `tile_id` should be correspondent to the tiles grid, and `band_name` should be one of 'B2', 'B3', 'B4', 'B8', 'B11', or 'B12'.
+ `path_to_samples_folder` (`string`)
    + A string path to the folder where training samples and auxiliary files should be stored.

In [ ]:
num_samples = 50000
samples_id = '001'
path_reference = '/path/to/reference.tif'
path_agriculture_mask = '/path/to/mask/of/agriculture.tif'
path_to_tiles_grid = '/path/to/tiles_grid.shp'
path_to_monthly_time_series_folder = '/path/to/monthly/time/series/folder' # f'Reduction_Optical_Months_id{tile_id:03}_{bands[j]}.tif'
path_to_samples_folder = '/path/to/samples/folder'

Next the reference data is opened and plotted. By doing this, one can confirm if data is adequate for use.

In [ ]:
ref = r.open(path_reference)
print('Reference dimensions:', ref.width, ref.height)

annual_perennial_reference = ref.read(1)

plt.figure(figsize=(10,10))
plt.imshow(annual_perennial_reference[::200,::200], interpolation='nearest', vmin=0, vmax=2)
plt.title('Annual and Perennial Reference')
plt.colorbar()
plt.show()

Now the agriculture mask is opened and plotted for inspection.

In [ ]:
agriculture_mask = r.open(path_agriculture_mask)
print('Agriculture mask dimensions:', agriculture_mask.width, agriculture_mask.height)

agriculture_mask = agriculture_mask.read(1)

plt.figure(figsize=(10,10))
plt.imshow(agriculture_mask[::200, ::200], interpolation='nearest', vmin=0, vmax=2)
plt.title('Agriculture Mask')
plt.show()

The following code masks the reference to match the agriculture mask used. It plots the result after it.

In [ ]:
annual_perennial_reference[agriculture_mask!=2] = 0

plt.figure(figsize=(10,10))
plt.imshow(annual_perennial_reference[::200, ::200], interpolation='nearest', vmin=0, vmax=2)
plt.title('Annual and Perennial Reference - Masked')
plt.show()

In [ ]:
print(f'Number of Temporary Crop Pixels: {np.sum(annual_perennial_reference==1):,}')
print(f'Number of Perennial Crops Pixels: {np.sum(annual_perennial_reference==2):,}')

The following code acquires all possible locations for annual and perennial training samples, and randomly selects training samples from them according to the number of desired samples defined previously.

In [ ]:
samples_annual = np.asarray(np.where(annual_perennial_reference==1)).T
np.random.shuffle(samples_annual)
samples_annual = samples_annual[:num_samples]

samples_perennial = np.asarray(np.where(annual_perennial_reference==2)).T
np.random.shuffle(samples_perennial)
samples_perennial = samples_perennial[:num_samples]

Now the locations of annual and perennial samples are merged and shuffled, do it does have to be done after extracting the samples time series.

In [ ]:
samples = np.zeros([num_samples*2,3], dtype=np.uint32)
samples[:num_samples, :-1] = samples_annual
samples[num_samples:, :-1] = samples_perennial

samples[:num_samples, -1] = 1
samples[num_samples:, -1] = 2

np.random.shuffle(samples)

In the next cell, the samples locations are converted from (row, columns) to (latitude, longitude). This allows the creation of a GeoDataframe, which is then displayed in order to check if the samples locations were chosen and converted appropriately.

In [ ]:
coords = np.asarray(r.transform.xy(transform=ref.transform, rows=samples[:,0], cols=samples[:,1], offset='center')).T
geo_points = [Point(coords[i]) for i in tqdm(range(len(coords)))]

samples_gdf = gpd.GeoDataFrame(samples, columns=['row', 'col', 'class'], geometry=geo_points)
samples_gdf.set_crs(crs="EPSG:4326", inplace=True)

samples_gdf.plot(column='class')
plt.show()

This is one of the most important parts of the algorithm, where the time series values of each sample are extracted. To do so, first it is decided within which tile the point it located, then this tile is opened according to the band wanted, then the point coordinates are used to sample over the time series tile. In the end a histogram is shown, so that the user can inspect the results and identify possible errors. For example, the sample placeholder was created with zero values, so it the histogram presents a suspiciously large amount of zeros, further investigations should be conducted. Please keep in mind that gaps in the time series are read with their nodata values, so large amounts of zeros can be normal also.

In [ ]:
samples_values = np.zeros([num_samples*2, 12, 6], dtype=np.float32)
samples_reference = samples[:,-1]

tiles = gpd.read_file(path_to_tiles_grid)
bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']

for i in tqdm(range(len(samples))):
    tile_id = int(tiles[tiles.intersects(geo_points[i])]['id'].values[0])
    for j in range(6):
        dataset = r.open(os.path.join(path_to_monthly_time_series_folder, f'Reduction_Optical_Months_id{tile_id:03}_{bands[j]}.tif'))
        samples_values[i, :, j] = np.asarray(list(dataset.sample([(geo_points[i].x, geo_points[i].y)])))

plt.hist(samples_values.ravel(), bins=20)
plt.show()

Now the extracted samples are stored using `numpy` file format (.npy).

In [ ]:
np.save(os.path.join(path_to_samples_folder, f'{samples_id}_raw.npy'), samples_values.astype(np.uint16))
np.save(os.path.join(path_to_samples_folder, f'{samples_id}_reference.npy'), samples_reference.astype(np.byte))

Limits for the samples scaling are obtained using percentiles equal to 2% (lower) and 98% (upper). The samples are then scaled according to each band and another histogram is shown for data inspection.

In [ ]:
limits = np.zeros([6,2], dtype=np.float32)

for i in tqdm(range(6)):
    limits[i] = np.percentile(samples_values[:,:,i][samples_values[:,:,i]!=0].ravel(), [2,98])

# scaling samples
for i in tqdm(range(6)):
    samples_values[:,:,i] = (samples_values[:,:,i]-limits[i,0])/(limits[i,1]-limits[i,0])

plt.hist(samples_values.ravel(), bins=100)
plt.show()

Scaled samples and the limits used for scaling are now stored for later use.

In [ ]:
np.save(os.path.join(path_to_samples_folder, f'{samples_id}_scaled.npy'), samples_values.astype(np.float32))
np.save(os.path.join(path_to_samples_folder, f'{samples_id}_limits.npy'), limits)

Stored data is opened and a histogram is plotted, in order to check for saving errors.

In [ ]:
test = np.load(os.path.join(path_to_samples_folder, f'{samples_id}_scaled.npy'))
plt.hist(test.ravel(), bins=1000)
plt.show()